# 1) Datasets

Vamos a leer el dataset que envía PyP

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
df = pd.read_csv('D:/MOPC/Enriquecimiento de Datos/Data Cleaning/Active/Datasets/Active_actual - 1101.csv')

In [ ]:
df.head()

In [ ]:
df.shape

## 1.2) Data Cleaning: Datasets

Vamos a dropear aquellas columnas y filas que no tengan ningún valor (es decir, que sean todos NaNs)

In [ ]:
df = df.dropna(axis=0, how='all')

In [ ]:
df = df.dropna(axis=1, how='all')

In [ ]:
df.shape

Ahora todas las columnas tienen al menos 1 dato.

# 2) Cleaning: Teléfonos

Verificar teléfonos if DF TIPO is TELEFONO

In [ ]:
df_telefonos = df[df['TIPO']== 'TELEFONO']

In [ ]:
df_telefonos.head()

In [ ]:
df_telefonos.shape

In [ ]:
df_telefonos['DATO'].value_counts().head(50)

Como regla incluso a nivel de pipeline: **todo lo que aparezca más de 10 veces es basura**.

In [ ]:
df_telefonos['DATO_CLEAN'] = [re.sub("[^0-9]", "", str(df_telefonos['DATO'][i])) for i in range (df_telefonos['DATO'].count())]

In [ ]:
df_telefonos['DATO_CLEAN'].value_counts().head(50)

## Condiciones

In [ ]:
#df_telefonos.loc[((df_telefonos['DATO_CLEAN'].str.len() >= 7)) , 'TELEFONO'] = df_telefonos['DATO_CLEAN']

In [ ]:
df_telefonos.tail()

Luego, que tenga al menos 2 caracteres diferentes. Por lo que nos quedamos con aquellos que tengan una len(set(str)) > 1

In [ ]:
a = '88888888888'

In [ ]:
len (set(a))

Defino una función para chequear los caracteres diferentes

In [ ]:
def distinct_character(string):
    dist_chars = len(set(string))
    if dist_chars > 1:
        ans = True
    else:
        ans = False
    return ans

Creo una columna temporal donde van los True/False

In [ ]:
df_telefonos['DIST_CHARACTER'] = df_telefonos['DATO_CLEAN'].apply(distinct_character) 

In [ ]:
df_telefonos.head()

In [ ]:
df_telefonos.loc[((df_telefonos['DIST_CHARACTER'] == True) & (df_telefonos['DATO_CLEAN'].str.len() >= 7)
            ) , 'TELEFONO'] = df_telefonos['DATO_CLEAN']

In [ ]:
df_telefonos.loc[((df_telefonos['DIST_CHARACTER'] == False) & (df_telefonos['DATO_CLEAN'].str.len() < 7)
                 ) , 'TELEFONO'] = np.nan

In [ ]:
df_telefonos.head()

In [ ]:
df_telefonos['TELEFONO'].value_counts().head(30)

In [ ]:
#df_telefonos[df_telefonos['IDMOROSO'] == 'COL 13747801      ']

## Drop

Dropeamos aquellos que tengan NaN en la columna Telefono

In [ ]:
df_telefonos = df_telefonos.dropna(subset=['TELEFONO'])

In [ ]:
#df_telefonos[df_telefonos['IDMOROSO'] == 'COL 13747801      ']

In [ ]:
df_telefonos.shape

In [ ]:
#df_telefonos = df_telefonos [['IDMOROSO','TELEFONO','TIPO','PROVEEDOR_DATO']]

## 12.2) DNI Cleaning

Limpiamos el dataframe __df_pyp__, ya que dentro de DATO_A_ENVIAR tenemos 9 tipos de casos diferentes de datos:

- DNI 12345678
- DNI12345678
- DNIE1234678
- LC  1234567
- LE  1234567
- CUIT00123456780
- 00123456780 --> Es un CUIT.
- 00 12345678 0 --> Es un CUIT.
- DNI 00123456780 --> Dice DNI pero es un CUIT de una persona __jurídica__ (TO DO)

Para eso definimos 5 funciones:

- __Tipo_DOC__: Se definen dos funciones para determinar el tipo de DNI. Una inicial, y otra para corregir sobre el dataframe resultante.
- __Standard DNI__: Estandariza los DNI para 7 de los casos. Esta función debe ejecutarse en primer lugar.
- __DNI Spaces__: Elimina los espacios.
- __DNI Zero__: Elimina los ceros iniciales de ciertos CUITs para que coincidan con los DNI.
- __Cuit__: Decisión sobre dejar un cuit o volverlo DNI

In [ ]:
def tipo_doc_1 (string_dni):
    
    if 'DNI' in string_dni:
        tipo = 'DNI'
    
    elif 'LC' in string_dni:
        tipo = 'LC'
    
    elif 'LE' in string_dni:
        tipo = 'LE'
    
    elif 'DNIE' in string_dni:
        tipo = 'DNIE'    
    
    elif ((string_dni.startswith(('CUIT30','CUIT33','CUIT34'))) & (len(string_dni)>8)):
        tipo = 'CUIT'
        
    elif ((string_dni.startswith(('30','33','34'))) & (len(string_dni)>8)):
        tipo = 'CUIT'

    elif ((string_dni.startswith(('20','23','24','27'))) & (len(string_dni)>8)):
        tipo = 'DNI'

    elif ((string_dni.startswith(('CUIT20','CUIT23','CUIT24','CUIT27'))) & (len(string_dni)>8)):
        tipo = 'DNI'
        
    else:
        tipo = 'DNI'
    
    return tipo

In [ ]:
def tipo_doc_2 (string_dni, tipo_original):
    
    if (len(string_dni) > 8) & (tipo_original == 'DNI'):
        tipo = 'CUIT'
    else:
        tipo = tipo_original
    return tipo

In [ ]:
def standard_dni (string_dni):
    """DNI separation by each case.
    """
    
    if 'DNI ' in string_dni:
        dni = string_dni.split('DNI ')[1]
        
    elif 'DNIE' in string_dni:
        dni = string_dni.split('DNIE')[1]
    
    elif 'DNI' in string_dni:
        dni = string_dni.split('DNI')[1]
    
    elif 'LC  ' in string_dni:
        dni = string_dni.split('LC  ')[1]
    
    elif 'LE  ' in string_dni:
        dni = string_dni.split('LE  ')[1]
    
    elif 'CUIT' in string_dni:
        dni = string_dni.split('CUIT')[1]
    
    elif 'COL ' in string_dni:
        dni = string_dni.split('COL ')[1]
        
    elif 'BBVA' in string_dni:
        dni = string_dni.split('BBVA')[1]
                
        
    else:
        dni = string_dni
        
    return dni

In [ ]:
def dni_characters (string_dni):
    
    if ' ' in string_dni:
        dni = string_dni.replace(' ','')
        
    elif '-' in string_dni:
        dni = string_dni.replace('-','')

    else:
        dni = string_dni
    
    return dni

In [ ]:
def cuit (string_dni):
    
    if ((string_dni.startswith('20')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]
        
    elif ((string_dni.startswith('23')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]
        
    elif ((string_dni.startswith('24')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]

    elif ((string_dni.startswith('27')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]

    else:
        cuit = string_dni
        
    return cuit

In [ ]:
def dni_zero (string_dni):
    
    if string_dni.startswith('0'):
        dni = string_dni [1:]

    else:
        dni = string_dni
    
    return dni

En primer lugar, trimmeamos los espacios que sobran:

In [ ]:
#df_telefonos = df_telefonos.dropna(subset=['IDCLIENTE'])

In [ ]:
df_telefonos ['IDCLIENTE'] = df_telefonos['IDCLIENTE'].str.strip()

In [ ]:
df_telefonos.loc[df_telefonos['IDMOROSO'].str.startswith('BSOL',na=False), 'NRO_DOC'] = df_telefonos['IDMOROSO'].str.slice(start=4)

In [ ]:
df_telefonos.loc[df_telefonos['IDMOROSO'].str.startswith('BRAIN',na=False), 'NRO_DOC'] = df_telefonos['IDMOROSO'].str.slice(start=5)

In [ ]:
prefijos = ('AMEX','CHUBB')

In [ ]:
df_telefonos.loc[(df_telefonos['IDCLIENTE'].str.startswith(prefijos, na=False)) , 'CUIT_CUIL'] = df_telefonos['DATOADIC']

In [ ]:
df_telefonos.loc[(df_telefonos['IDCLIENTE'].str.startswith(prefijos, na=False)), 'NRO_DOC'] = df_telefonos['DATOADIC']

In [ ]:
df_telefonos.loc[(~df_telefonos['IDCLIENTE'].str.startswith(prefijos, na=False)), 'NRO_DOC'] = df_telefonos['IDMOROSO']

In [ ]:
df_telefonos = df_telefonos.astype({'NRO_DOC': 'str'})

In [ ]:
df_telefonos['NRO_DOC'] = df_telefonos['NRO_DOC'].str.strip()

Se implementan estas funciones:

Para determinar el __TIPO DE DOCUMENTO__ en primer lugar:

In [ ]:
df_telefonos['TIPO_DOC'] = df_telefonos['NRO_DOC'].apply(tipo_doc_1)

Para la limpieza de DNIs:

In [ ]:
df_telefonos['NRO_DOC'] = df_telefonos['NRO_DOC'].apply(standard_dni)

In [ ]:
df_telefonos['NRO_DOC'] = df_telefonos['NRO_DOC'].apply(dni_characters)

In [ ]:
df_telefonos['NRO_DOC'] = df_telefonos['NRO_DOC'].apply(cuit)

In [ ]:
df_telefonos['NRO_DOC'] = df_telefonos['NRO_DOC'].apply(dni_zero)

Para corregir el __TIPO DE DOCUMENTO__ sobre el dataframe resultante

In [ ]:
# REVISAR

df_telefonos['TIPO_DOC'] = df_telefonos.apply(lambda x: tipo_doc_2(x['NRO_DOC'], x['TIPO_DOC']), axis=1)

In [ ]:
df_telefonos.head()

## Campos adicionales

Asigno campos adicionales que no tengo

In [ ]:
df_telefonos = df_telefonos.assign(SUBTIPO='PERSONAL')

In [ ]:
df_telefonos = df_telefonos.astype({'TELEFONO': 'str'})

In [ ]:
df_telefonos.loc[((df_telefonos['TELEFONO'].str.startswith('11')) |((df_telefonos['TELEFONO'].str.startswith('15'))
            )) , 'TIPO'] = 'TEL.CELULAR'

In [ ]:
df_telefonos.loc[(df_telefonos['TIPO'] != 'TEL.CELULAR') , 'TIPO'] = 'TEL.FIJO'

In [ ]:
df_telefonos = df_telefonos.rename(columns = {'TELEFONO':'DATO_a_ingresar'})

In [ ]:
df_telefonos = df_telefonos.assign(PROPIETARIO_DATO='Propio')

In [ ]:
df_telefonos = df_telefonos.assign(FECHA_INGRESO_DATO=pd.to_datetime('today'))

In [ ]:
df_telefonos.loc[df_telefonos['VERIFICADO'] == 'S', 'VERIFICADO'] = True

In [ ]:
df_telefonos.loc[df_telefonos['VERIFICADO'] == 'N', 'VERIFICADO'] = False

In [ ]:
#df_telefonos['FECHA_VERIFICACION'] = np.nan

In [ ]:
df_telefonos['OPERADOR_VERIFICACION'] = np.nan

In [ ]:
df_telefonos['SCORE_DATO'] = 4

In [ ]:
df_telefonos.head()

# Export

In [ ]:
df_telefonos.shape

In [ ]:
df_export = df_telefonos.drop_duplicates()

In [ ]:
df_export.shape

In [ ]:
df_export = df_export[['IDMOROSO','TIPO_DOC','NRO_DOC','CUIT_CUIL','TIPO','SUBTIPO','DATO_a_ingresar','PROPIETARIO_DATO','PROVEEDOR_DATO','FECHA_INGRESO_DATO','VERIFICADO','FECHA_VERIFICACION','OPERADOR_VERIFICACION','SCORE_DATO']]

In [ ]:
df_export = df_export.rename(columns={'DATO_a_ingresar':'DATO'})

In [ ]:
df_export = df_export.drop_duplicates()

In [ ]:
df_export[df_export.duplicated(subset=['IDMOROSO','DATO','PROVEEDOR_DATO'],keep=False)]

In [ ]:
duplicates = df_export [df_export.duplicated(subset=['IDMOROSO','DATO'],keep=False)]

In [ ]:
duplicates.shape

In [ ]:
#duplicates.to_csv('dups_telefonos.csv')

In [ ]:
df_export_at = df_export

### Post pipeline Duplicados

In [ ]:
subset_add_1 = pd.read_csv('D:\MOPC\Enriquecimiento de Datos\Data Cleaning\Active\Exports\DAM_CONTACTABILIDAD\telefonos_non-dups.csv')

In [ ]:
subset_add_1.head()

In [ ]:
non_duplicates = df_export_at[~df_export_at.duplicated(subset=['IDMOROSO','DATO','PROVEEDOR_DATO'],keep=False)]

In [ ]:
subset_add_1 = subset_add_1.astype({'NRO_DOC': 'str'})

In [ ]:
subset_add_1['NRO_DOC'] = subset_add_1['NRO_DOC'].str.split('.').str[0]

In [ ]:
df_export_tels = pd.concat([subset_add_1,non_duplicates],ignore_index=True)

In [ ]:
df_export_tels.shape

In [ ]:
df_export_tels[df_export_tels.duplicated(subset=['IDMOROSO','DATO'],keep = False)]

## Final Cleaning

IF BSOL en NRO_DOC, trimmear los primeros 4 caracteres.

In [ ]:
#df_export_mails.loc[df_export_mails['IDMOROSO'].str.startswith('BSOL',na=False), 'NRO_DOC'] = df_export_mails['IDMOROSO'].str.slice(start=4)

In [ ]:
df_export_tels.loc[df_export_tels['NRO_DOC'].str.startswith('Ing',na=False), 'NRO_DOC'] = np.nan

In [ ]:
df_export_tels = df_export_tels.astype({'CUIT_CUIL': 'str'})

In [ ]:
df_export_tels['CUIT_CUIL'] = df_export_tels['CUIT_CUIL'].str.strip()

In [ ]:
df_export_tels.loc[df_export_tels['CUIT_CUIL'].str.startswith('Ing',na=False), 'CUIT_CUIL'] = np.nan

In [ ]:
len(df_export_tels['CUIT_CUIL'][132678])

In [ ]:
df_export_tels[df_export_tels['IDMOROSO']== 'RMS376408462261000']

In [ ]:
#df_export_tels.loc[df_export_tels['CUIT_CUIL'].str.len() < 11, 'CUIT_CUIL'] = np.nan

In [ ]:
df_export_tels.loc[df_export_tels['IDMOROSO'].str.startswith('CUIT',na=False), 'CUIT_CUIL'] = df_export_tels['NRO_DOC']

In [ ]:
df_export_tels[df_export_tels['IDMOROSO']== 'RMS376408462261000']

In [ ]:
df_export_tels[df_export_tels.index.duplicated()] #indices duplicados al hacer el concat

In [ ]:
df_export_tels.loc[df_export_tels['NRO_DOC'].str.contains('$',regex=False,na=False), 'NRO_DOC'] = np.nan

In [ ]:
df_export_tels.loc[df_export_tels['NRO_DOC'].str.contains('-',regex=False,na=False), 'NRO_DOC'] = np.nan

In [ ]:
df_export_tels.loc[df_export_tels['CUIT_CUIL'].str.contains('$',regex=False,na=False), 'CUIT_CUIL'] = np.nan

In [ ]:
df_export_tels.loc[df_export_tels['CUIT_CUIL'].str.contains('-',regex=False,na=False), 'CUIT_CUIL'] = np.nan

In [ ]:
df_export_tels.loc[df_export_tels['NRO_DOC'] == 'nan', 'NRO_DOC'] = np.nan

In [ ]:
df_export_tels.loc[df_export_tels['CUIT_CUIL'] == 'nan', 'CUIT_CUIL'] = np.nan

In [ ]:
prefijos_dni = ('DNIF','DNIM')

In [ ]:
df_export_tels.loc[df_export_tels['IDMOROSO'].str.startswith(prefijos_dni,na=False), 'TIPO_DOC'] = 'DNI'

In [ ]:
def subst_char (string):
    x = re.sub("[^0-9]", "", str(string))
    return x

In [ ]:
df_export_tels['NRO_DOC'] = df_export_tels.apply(lambda x: subst_char(x['NRO_DOC']), axis=1)

In [ ]:
df_export_tels['CUIT_CUIL'] = df_export_tels.apply(lambda x: subst_char(x['CUIT_CUIL']), axis=1)

In [ ]:
df_export_tels.shape

In [ ]:
df_export_tels.loc[df_export_tels['CUIT_CUIL'].str.len() < 11, 'CUIT_CUIL'] = np.nan

In [ ]:
df_export_tels.to_csv('D:\MOPC\Enriquecimiento de Datos\Data Cleaning\Active\Exports\DAM_CONTACTABILIDAD\AT_DAM_CONTACTABILIDAD_telefonos_1501.csv',index=False)

In [ ]:
#df_export_tels.to_csv('AT_DAM_CONTACTABILIDAD_telefonos_1401.csv',index=False)

In [ ]:
df_export_tels[df_export_tels['IDMOROSO'] == 'DNIE94301664      ']